In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys
import os

In [3]:
df = pd.read_csv("../../../data/raw/raw_bhp.csv")
df.head(2)

,Name,Operating-System,Processor,GPU,Total-Installed-Memory,Display-Size,Resolution,Touch-Screen,Total-Installed-Capacity,Inputs/Outputs,...,Batery-Chemistry,Capacity,Keyboard,Power-Supply,Operating-Tempurature,Operating-Humidity,Dimensions,Package-Weight,Box-Dimension (LxWxH),Price
0,"ASUS 13.3"" Zenbook S 13 OLED Laptop (Basalt Gray)",Windows 11 Pro,Intel Core i7-1355U (13th Gen),Intel Iris Xe,32 GB,"13.3""",2880 x 1800,No,1 TB,1 x USB-A 3.1/3.2 Gen 22 x USB-C (Thunderbolt ...,...,NaN,4-Cell: 63 Wh,NaN,65 W with USB-C,-60 to 160 F / -51 to 71 C,95%,"11.66 x 8.52 x 0.46"" / 29.62 x 21.63 x 1.18 cm",5.895 lb,"17.8 x 11.1 x 4""","$1,299.99"
1,"ASUS 15.6"" Vivobook S 15 OLED Laptop (Midnight...",Windows 11 Home,Intel Core i9-13900H (13th Gen),Intel Iris Xe,16 GB,"15.6""",2880 x 1620,No,1 TB,1 x USB-A 3.0 / 3.1/3.2 Gen 11 x USB-A 2.01 x ...,...,NaN,4-Cell: 75 Wh,NaN,65 W with Barrel/Proprietary,-6 to 109 F / -21 to 43 C,95%,"14.15 x 9.03 x 0.7"" / 35.93 x 22.94 x 1.79 cm",5.905 lb,"18.2 x 11.6 x 2.4""",$989.99


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1420 entries, 0 to 1419
Data columns (total 53 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Name                       1420 non-null   object
 1   Operating-System           1378 non-null   object
 2   Processor                  1379 non-null   object
 3   GPU                        1380 non-null   object
 4   Total-Installed-Memory     1381 non-null   object
 5   Display-Size               1381 non-null   object
 6   Resolution                 1381 non-null   object
 7   Touch-Screen               1380 non-null   object
 8   Total-Installed-Capacity   1381 non-null   object
 9   Inputs/Outputs             1381 non-null   object
 10  Display-I/O                1023 non-null   object
 11  WiFi                       1381 non-null   object
 12  Bluetooth                  1379 non-null   object
 13  Weight                     1370 non-null   object
 14  Model-Ye

In [5]:
df.isna().sum()

Name                            0
Operating-System               42
Processor                      41
GPU                            40
Total-Installed-Memory         39
Display-Size                   39
Resolution                     39
Touch-Screen                   40
Total-Installed-Capacity       39
Inputs/Outputs                 39
Display-I/O                   397
WiFi                           39
Bluetooth                      41
Weight                         50
Model-Year                    919
CPU                            39
L3-Cache                       45
Graphics-Type                 370
Memory-Type                    39
Memory-Configuration           39
Maximum-Memory-Capacity       961
Memory-Slots                  960
ECC-Memory                    399
vPro-Support                  272
Panel-Type                     39
Aspect-Ratio                   39
Finish                        330
Maximum-Brightness            218
Column-Gamut                  689
Refresh-Rate  

In [7]:
merge_df = pd.read_csv("../../../data/processed/merge_dataset.csv")

In [8]:
merge_df["OS"].value_counts()

Windows 11             4783
Windows 10             2785
Chrome OS               152
ChromeOS                136
No OS                    47
Windows 7                28
Windows 8                14
MacOS                     8
Windows 8.1               7
Google Chrome             5
Linux                     3
Windows Vista             3
Windows 1                 2
OS X 10.10 Yosemite       1
Windows 1012              1
Windows 14                1
Windows Pro               1
Windows 108               1
Ubuntu Linux 20.04        1
Windows 1111              1
Ubuntu Linux 18.04        1
Name: OS, dtype: int64

In [103]:
df_copy = df
df_copy['Operating-System'] = df_copy['Operating-System'].replace({
    'Windows 11 Pro (64-Bit)': 'Windows 11 Pro',
    'Windows 11 Pro for Workstations (64-Bit)': 'Windows 11 Pro',

    'Windows 11 Home (64-Bit)': 'Windows 11 Home',
    'Windows 11 Home in S Mode': 'Windows 11 Home',

    'Windows 10 Pro (64-Bit)': 'Windows 10 Pro',
    'Windows 10 Pro Education': 'Windows 10 Pro',

    'Windows 10 Home in S Mode': 'Windows 10 Home',
    'Windows 11 SE (64-Bit)': 'Windows 10 Home',
    'Windows 10 IoT Enterprise (64-Bit)': 'Windows 10 Home',
    'Dell ThinOS (64-Bit)': 'Windows 10 Home',

    'Chrome OS Enterprise': 'Chrome OS',
    'Chrome OS (64-Bit)': 'Chrome OS',
})

df_copy.loc[df_copy['Operating-System'].isna() & df_copy['Name'].str.startswith('Apple'), 'Operating-System'] = 'macOS'
df_copy.loc[df_copy['Operating-System'].isna() & ~df_copy['Name'].str.startswith('Apple'), 'Operating-System'] = 'Windows 11 Home'

df_copy['Operating-System'].value_counts()

Windows 11 Pro     781
macOS              349
Windows 11 Home    169
Chrome OS           61
Windows 10 Pro      52
Windows 10 Home      8
Name: Operating-System, dtype: int64

In [104]:
df_copy = df_copy.drop(labels=["Box-Dimension (LxWxH)", "Dimensions", "Power-Supply", "Keyboard", "GPS", "Celluar-Support", "Media/Memory Card Slot", "Built-In-Microphones", "Built-In-Speakers", "Audio-I/O", "Variable-Fresh-Technology", "Contrast-Ratio", "Finish", "Aspect-Ratio", "Column-Gamut", "vPro-Support", "ECC-Memory", "Model-Year", "WiFi", "Display-I/O", "Inputs/Outputs"], axis=1)
df_copy["Brand"] = df_copy["Name"].str.split().apply(lambda x: x[0])

In [105]:
df_copy.drop(["Touch-Screen", "Bluetooth", "Storage-Expansion", "Network-I/O", "Webcam", "Batery-Chemistry", "Capacity", "Operating-Tempurature", "Operating-Humidity", "Package-Weight"], axis=1, inplace=True)

In [106]:
df_copy.drop(["Storage-Drive-Type", "Response-Time", "Maximum-Brightness", "Panel-Type", "Memory-Slots", "Maximum-Memory-Capacity"], axis=1, inplace=True)

In [107]:
df_copy.drop(["L3-Cache", "Graphics-Type", "Memory-Type"], axis=1, inplace=True)

In [108]:
df_copy["Price"] = df_copy["Price"].replace('[\$,]', '', regex=True).astype(float)
df_copy["Price"].fillna(df_copy["Price"].mean(), inplace=True)

df_copy["Price"].value_counts()

1989.312761    29
1899.000000    26
1599.000000    25
1299.000000    23
1699.000000    23
               ..
2258.070000     1
1077.500000     1
1925.830000     1
1944.500000     1
2249.000000     1
Name: Price, Length: 782, dtype: int64

In [109]:
df_copy["Total-Installed-Memory"].fillna(value="8 GB", inplace=True)
df_copy["Total-Installed-Memory"] = df_copy["Total-Installed-Memory"].str.split().apply(lambda x: x[0]).astype(int)

In [110]:
df_copy["Display-Size"].fillna("14\"", inplace=True)
df_copy["Display-Size"] = df_copy["Display-Size"].apply(lambda x: x[:-1]).astype(float)

In [111]:
df_copy["Resolution"].fillna("1920 x 1080", inplace=True)
df_copy["Width"] = df_copy["Resolution"].str.split().apply(lambda x: x[0]).astype(int)
df_copy["Height"] = df_copy["Resolution"].str.split().apply(lambda x: x[-1]).astype(int)
df_copy = df_copy.drop("Resolution", axis=1)

In [112]:
def convert_memory(amount: str) -> int:
    value_, type_ = amount.split()
    if type_ == "TB":
        return int(value_) * 1024
    
    if type_ == "GB":
        return int(value_)
    
df_copy["Total-Installed-Capacity"].fillna("512 GB", inplace=True)
df_copy["Total-Installed-Capacity"] = df_copy["Total-Installed-Capacity"].apply(convert_memory)

In [113]:
df_copy['Weight'] = df_copy['Weight'].str.split('/').str[1].str.strip().str.rstrip('kg').str.strip().astype(float)
mean_weight = df_copy['Weight'].mean()
df_copy['Weight'] = df_copy['Weight'].fillna(mean_weight)

In [114]:
df_copy.drop(["Refresh-Rate", "Memory-Configuration"], axis=1, inplace=True)

In [115]:
df_copy

,Name,Operating-System,Processor,GPU,Total-Installed-Memory,Display-Size,Total-Installed-Capacity,Weight,CPU,Price,Brand,Width,Height
0,"ASUS 13.3"" Zenbook S 13 OLED Laptop (Basalt Gray)",Windows 11 Pro,Intel Core i7-1355U (13th Gen),Intel Iris Xe,32,13.3,1024,1.00,10-Core: 1.7 to 5 GHz Performance (2 Cores)1.2...,1299.990000,ASUS,2880,1800
1,"ASUS 15.6"" Vivobook S 15 OLED Laptop (Midnight...",Windows 11 Home,Intel Core i9-13900H (13th Gen),Intel Iris Xe,16,15.6,1024,1.70,14-Core: 2.6 to 5.4 GHz Performance (6 Cores)1...,989.990000,ASUS,2880,1620
2,"ASUS 15.6"" Vivobook Laptop (Quiet Blue)",Windows 11 Home,AMD Ryzen 7 7730U,AMD Radeon,16,15.6,1024,1.70,8-Core: 2 to 4.5 GHz,699.990000,ASUS,1920,1080
3,"ASUS 16"" ProArt StudioBook Pro 16 OLED Multi-T...",Windows 11 Pro,Intel Core i9-13980HX (13th Gen),NVIDIA RTX 3000 Ada (130 W TGP) with 8 GB GDD...,64,16.0,2048,2.40,24-Core: 2.2 to 5.6 GHz Performance (8 Cores)1...,2999.990000,ASUS,3200,2000
4,"Lenovo 15.6"" V15 G4 IAN Laptop (Business Black)",Windows 11 Home,Intel N100,Intel UHD Graphics,8,15.6,128,1.65,Quad-Core: Up to 3.4 GHz,367.500000,Lenovo,1920,1080
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,"HP 15.6"" EliteBook 650 G10 Multi-Touch Noteboo...",Windows 11 Pro,Intel Core i7-1355U (13th Gen),Intel Iris Xe,16,15.6,256,1.79,10-Core: 1.7 to 5 GHz Performance (2 Cores)1.2...,1989.312761,HP,1920,1080
1416,"ASUS 14.5"" Zenbook Pro 14 Duo OLED Multi-Touch...",Windows 11 Home,Intel Core i9-13900H (13th Gen),NVIDIA GeForce RTX 4060 with 8 GB GDDR6 VRAM,32,14.5,1024,1.75,14-Core: 2.6 to 5.4 GHz Performance (6 Cores)1...,2299.000000,ASUS,2880,1800
1417,"ASUS 16"" Republic of Gamers Zephyrus Duo 16 Ga...",Windows 11 Pro,AMD Ryzen 9 7945HX,NVIDIA GeForce RTX 4090 (175 W TGP) with 16 G...,32,16.0,2048,2.67,16-Core: 2.5 to 5.4 GHz,3799.000000,ASUS,2560,1600
1418,"Lenovo 16"" ThinkPad P1 Gen 5 Notebook with Pre...",Windows 11 Pro,Intel Core i7-12800H (12th Gen),NVIDIA RTX A4500 with 16 GB GDDR6 VRAM,16,16.0,512,1.81,14-Core: 2.4 to 4.8 GHz Performance (6 Cores)1...,2499.000000,Lenovo,2560,1600


In [116]:
bhp_df = df_copy

In [117]:
bhp_df[["Processor", "CPU", "GPU"]].head(5)

,Processor,CPU,GPU
0,Intel Core i7-1355U (13th Gen),10-Core: 1.7 to 5 GHz Performance (2 Cores)1.2...,Intel Iris Xe
1,Intel Core i9-13900H (13th Gen),14-Core: 2.6 to 5.4 GHz Performance (6 Cores)1...,Intel Iris Xe
2,AMD Ryzen 7 7730U,8-Core: 2 to 4.5 GHz,AMD Radeon
3,Intel Core i9-13980HX (13th Gen),24-Core: 2.2 to 5.6 GHz Performance (8 Cores)1...,NVIDIA RTX 3000 Ada (130 W TGP) with 8 GB GDD...
4,Intel N100,Quad-Core: Up to 3.4 GHz,Intel UHD Graphics


In [118]:
cpu_df = pd.read_csv('../../../data/raw/cpu_gpu_mark/cpu_mark.csv', index_col = 0)
gpu_df = pd.read_csv('../../../data/raw/cpu_gpu_mark/gpu_mark.csv', index_col = 0)

cpu_name_list = [cpu_df['CPU Name'][idx].lower() for idx in cpu_df.index]
gpu_name_list = [gpu_df['GPU Name'][idx].lower() for idx in gpu_df.index]

In [119]:
import numpy as np

In [120]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
 
def mapping(s, slist):
    s2 = s.lower()
    found=[0, -1]
    for i in range(len(slist)):
        if s2 in slist[i]:
            found[0] = 100
            found[1] = i
            break
        acc=sum([fuzz.ratio(slist[i], s2),fuzz.partial_ratio(slist[i], s2),fuzz.token_sort_ratio(slist[i], s2),fuzz.token_set_ratio(slist[i], s2)])/4
        if found[0]<acc:
            found[0]=acc
            found[1]=i
    return found
 
def transform_cpu(row):
    cpu = row['Processor_']
    if cpu:
        cpu=cpu.lower()
        acc, pos= mapping(cpu, cpu_name_list)
        if acc>50:
            row["CPU Mark"] = cpu_df['CPU Rank'][pos]
        else:
            row["CPU Mark"] = np.nan
    return row
 
def transform_gpu(row):
    gpu = row['GPU']
    if gpu:
        gpu=gpu.lower()
        acc, pos=mapping(gpu, gpu_name_list)
        if acc>50:
            row["GPU Mark"] = gpu_df['GPU Rank'][pos]
        else:
            row["GPU Mark"] = np.nan
    return row

In [121]:
def fill_processor(row):
    if pd.isna(row["Processor"]):
        if row["Brand"].lower() == "apple":
            return "Apple M2"
        else:
            return "Intel Core i5-1335U (13th Gen)"
    else:
        return row["Processor"]

bhp_df["Processor_"] = bhp_df.apply(fill_processor, axis=1)

In [122]:
bhp_df["Processor"].value_counts()

Apple M2                           126
Apple M3 Max                        98
Intel Core i5-1335U (13th Gen)      87
Intel Core i7-1355U (13th Gen)      77
Intel Core i7-1360P (13th Gen)      65
                                  ... 
Intel Celeron N3350                  1
AMD Ryzen 5-7530U                    1
AMD Ryzen 7 PRO 6850H                1
Intel Core i7-12700H (12th Gen)      1
Apple M1 Pro                         1
Name: Processor, Length: 119, dtype: int64

In [123]:
bhp_df = bhp_df.apply(transform_cpu, axis=1)
bhp_df.isna().sum()

Name                         0
Operating-System             0
Processor                   41
GPU                         40
Total-Installed-Memory       0
Display-Size                 0
Total-Installed-Capacity     0
Weight                       0
CPU                         39
Price                        0
Brand                        0
Width                        0
Height                       0
Processor_                   0
CPU Mark                     0
dtype: int64

In [124]:
bhp_df.dropna(subset="GPU", inplace=True)

In [125]:
bhp_df = bhp_df.apply(transform_gpu, axis=1)

In [126]:
bhp_df.isna().sum()

Name                          0
Operating-System              0
Processor                     1
GPU                           0
Total-Installed-Memory        0
Display-Size                  0
Total-Installed-Capacity      0
Weight                        0
CPU                           0
Price                         0
Brand                         0
Width                         0
Height                        0
Processor_                    0
CPU Mark                      0
GPU Mark                    299
dtype: int64

In [127]:
bhp_df.describe()

,Total-Installed-Memory,Display-Size,Total-Installed-Capacity,Weight,Price,Width,Height,CPU Mark,GPU Mark
count,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1380.000000,1081.000000
mean,22.942754,14.687101,881.786957,6.710217,1998.179935,2314.943478,1439.150725,20293.028986,7214.954672
std,20.282674,1.245316,1160.729239,70.344796,1193.167985,586.103565,412.518718,9353.090604,9408.649597
min,3.000000,11.600000,2.000000,0.800000,99.000000,1280.000000,720.000000,787.000000,124.000000
25%,16.000000,14.000000,256.000000,1.337500,1243.957500,1920.000000,1080.000000,15348.000000,1971.000000
50%,16.000000,14.200000,512.000000,1.560000,1733.625000,1920.000000,1200.000000,16980.000000,1971.000000
75%,32.000000,16.000000,1024.000000,1.840000,2399.000000,2560.000000,1664.000000,26364.000000,12961.000000
max,128.000000,18.000000,8192.000000,991.500000,7199.000000,3840.000000,2400.000000,55651.000000,38874.000000


In [131]:
bhp_df.to_csv("processed_bhp.csv")

In [130]:
bhp_df.isna().sum()

Name                          0
Operating-System              0
Processor                     1
GPU                           0
Total-Installed-Memory        0
Display-Size                  0
Total-Installed-Capacity      0
Weight                        0
CPU                           0
Price                         0
Brand                         0
Width                         0
Height                        0
Processor_                    0
CPU Mark                      0
GPU Mark                    299
dtype: int64

In [129]:
merge_df.drop(merge_df.columns[0], axis=1, inplace=True)
merge_df

,CPU Name,CPU Mark,GPU Name,GPU Mark,Monitor,Width,Height,RAM,Storage Amount,OS,Weight,Price
0,Intel Core i7 12700H,26423.0,Intel Iris Xe,2690.0,15.6,1920,1080,40.0,2048.0,Windows 11,1.77,1199.99
1,Intel Core i5 750S 2.40GHz,1326.0,Intel HD Graphics 620,926.0,14.0,1920,1080,8.0,128.0,Windows 10,1.55,1248.00
2,Intel Core i5 1145G7 2.60GHz,9970.0,Intel Iris Xe,2690.0,13.5,2256,1504,8.0,512.0,Windows 10,1.27,1381.99
3,Intel Core i5 1135G7 2.40GHz,9891.0,GeForce MX450,3729.0,15.6,1920,1080,16.0,1024.0,Windows 11,1.65,549.00
4,Intel Core i5 1240P,17329.0,Intel Arc,4253.0,13.3,1920,1080,8.0,256.0,Windows 11,0.86,838.05
...,...,...,...,...,...,...,...,...,...,...,...,...
7976,Qualcomm,1354.0,Qualcomm Adreno 618 GPU,124.0,13.3,1366,768,4.0,128.0,Chrome OS,1.21,530.99
7977,Intel Core i7 13705H,23728.0,GeForce RTX 3050,12961.0,15.6,1920,1200,16.0,2000.0,Windows 11,3.64,1971.58
7978,Intel Core i3 1115G4 3.00GHz,6132.0,Intel HD Graphics 620,926.0,15.6,1920,1080,16.0,512.0,Windows 10,1.77,499.99
7979,AMD Ryzen 5 PRO 5650U,14922.0,Radeon Ryzen 5 4600GE with Radeon Graphics,1792.0,14.0,1920,1080,8.0,256.0,Windows 10,2.33,556.99
